# Поиск каверов композиций
<br>Разработка ML-модели для сопоставления текстов музыкальных произведений и для поиска каверов (вариации обработки оригинала с элементами новой аранжировки) по их текстам

<br>**Описание проекта**
<br>Обнаружение треков каверов - важная продуктовая задача, которая может значительно улучшить качество рекомендаций музыкального сервиса и повысить счастье наших пользователей.
<br>Если мы умеем с высокой точностью классифицировать каверы и связывать их между собой, то можно предложить пользователю новые возможности для управления потоком треков.
<br>Например:
- по желанию пользователя можем полностью исключить каверы из рекомендаций;
- показать все каверы на любимый трек пользователя;
- контролировать долю каверов в ленте пользователя.

<br>**Цель проекта**
<br>Необходимо разработать ML-продукт, который:
- классифицирует треки по признаку кавер-некавер;
- связывает (группирует) каверы и исходный трек;
- находит исходный трек в цепочке каверов.

<br>**Описание данных**
- Файл covers.json содержит разметку каверов, сделанную редакторами сервиса:
    - track_id - уникальный идентификатор трека;
    - track_remake_type - метка, присвоенная редакторами. Может принимать значения ORIGINAL и COVER;
    - original_track_id - уникальный идентификатор исходного трека.
- Метаинформация:
    - track_id - уникальный идентификатор трека;
    - dttm - первая дата появления информации о треке;
    - title - название трека;
    - language - язык исполнения;
    - isrc - международный уникальный идентификатор трека;
    - genres - жанры;
    - duration - длительность трека.
- Текст песен:
    - track_id - уникальный идентификатор трека;
    - lyricId - уникальный идентификатор текста;
    - text - текст трека.

<br>**Целевой метрикой выбрана** `f1`

## Импорт библиотек

In [3]:
# !pip install sentence-transformers #-U
# !pip install lyricsgenius
# !pip install gensim
# drive.mount('/content/drive')

In [3]:
# from google.colab import drive
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import xgboost
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# from xgboost import XGBClassifier
# from MusicMetaLinker.MusicMetaLinker.linking import linking
# import gensim
# from lyricsgenius import Genius
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import f1_score
# sklearn, CatBoost, XGBoost, LightGBM, NLP
warnings.filterwarnings('ignore')

Установка констант

In [4]:
RS = 42 # random state

## EDA

### Загрузка данных

In [5]:
try:
    drive.mount('/content/drive')
    covers = pd.read_json('./drive/MyDrive/Colab Notebooks/music_covers/covers.json', orient='records', lines=True)
    lyrics = pd.read_json('./drive/MyDrive/Colab Notebooks/music_covers/lyrics.json', orient='records', lines=True)
    meta = pd.read_json('./drive/MyDrive/Colab Notebooks/music_covers/meta.json', orient='records', lines=True)
    print('загружено из каталога "Colab Notebooks"')
except:
    covers = pd.read_json('data/covers.json', orient='records', lines=True)
    lyrics = pd.read_json('data/lyrics.json', orient='records', lines=True)
    meta = pd.read_json('data/meta.json', orient='records', lines=True)
    print('загружено из каталога "data"')

np.array(covers).shape, np.array(lyrics).shape, np.array(meta).shape

загружено из каталога "data"


((71597, 3), (11414, 3), (71769, 7))

### Датафрейм `lyrics`

In [6]:
lyrics.info()
display(lyrics.sample(3))
print('явные повторы строк:', lyrics.duplicated().sum())
display('количество уникальных записей:', lyrics.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11414 entries, 0 to 11413
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   lyricId   11414 non-null  object
 1   text      11414 non-null  object
 2   track_id  11414 non-null  object
dtypes: object(3)
memory usage: 267.6+ KB


,lyricId,text,track_id
7949,1219c0cb2c4738d744eda6007fa17e62,"Been there done that messed around,\nI'm havin...",ecfcc3542d9219bf01cd5fdf92821b79
10448,b83f3d7cd6ba327e56f362b3231b20af,"I come home, in the mornin' light\nMy mother s...",81fc425f0de19fc76cedb36a6064a894
9530,fcafc3120bc04b44b0ffb288b010fc14,"Молчи, когда бухой сосед на свою тёлку кричит\...",e9f2d002484a165a59aba757bfab3f3d


явные повторы строк: 0


'количество уникальных записей:'

lyricId     10915
text        10644
track_id    10277
dtype: int64

Выводы:
- тексты представлены на различных языках (английский, русский, испанский)
- явных повторов строк не обнаружено, при этом количество уникальных ID текстов и треков меньше,
  <br>чем всего строк, что означает использование текстов в других треках

### Датафрейм `meta`

In [7]:
meta.info()
display(meta.sample()) #genres - list
print('явные повторы строк:', meta[['track_id', 'dttm', 'title', 'language', 'isrc', 'duration']].duplicated().sum())
display('количество уникальных записей:', meta[['track_id', 'dttm', 'title', 'language', 'isrc', 'duration']].nunique())
display('список представленных стран:', meta['language'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71769 entries, 0 to 71768
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   track_id  71768 non-null  object 
 1   dttm      71768 non-null  float64
 2   title     71768 non-null  object 
 3   language  21969 non-null  object 
 4   isrc      71455 non-null  object 
 5   genres    71768 non-null  object 
 6   duration  71768 non-null  float64
dtypes: float64(2), object(5)
memory usage: 3.8+ MB


,track_id,dttm,title,language,isrc,genres,duration
25016,416e20724e2848b8e92e9583bb5d3c79,1.623876e+12,The Kids Aren't Alright,None,QZHZ52183036,"[ROCK, ALLROCK]",186300.0


явные повторы строк: 0


'количество уникальных записей:'

track_id    71768
dttm        27726
title       45462
language       85
isrc        71283
duration    23597
dtype: int64

'список представленных стран:'

array(['EN', None, 'ES', 'HI', 'DE', 'RU', 'TR', 'HU', 'TH', 'PL', 'FR',
       'NY', 'AF', 'AS', 'UZ', 'HT', 'EL', 'AZ', 'IT', 'PA', 'PT', 'TA',
       'JA', 'ML', 'VI', 'ID', 'LA', 'CS', 'SI', 'UK', 'OR', 'HR', 'AR',
       'KK', 'FI', 'IE', 'ZH', 'AB', 'KN', 'FA', 'BN', 'TL', 'SK', 'KS',
       'SV', 'RO', 'TN', 'KO', 'MS', 'BM', 'HY', 'TW', 'MY', 'CA', 'NL',
       'ET', 'TE', 'MN', 'HE', 'SQ', 'IG', 'MR', 'BE', 'LT', 'UR', 'IA',
       'GN', 'SW', 'NO', 'GU', 'KY', 'KU', 'IS', 'TG', 'SR', 'DA', 'LO',
       'LV', 'SE', 'WO', 'SA', 'YO', 'ST', 'HA', 'AV', 'IU'], dtype=object)

In [ ]:
# from datetime import datetime

# unix_timestamp1 = 1.555760e+12
# unix_timestamp2 = 1.626110e+12

# # Преобразование временной метки в объект datetime
# dt1 = datetime.fromtimestamp(unix_timestamp1 / 1000) #2019-04-20 14:33:20
# dt2 = datetime.fromtimestamp(unix_timestamp2 / 1000) #2021-07-12 20:13:20

# print(dt1 < dt2) #True
# unix_timestamp1 < unix_timestamp2 #True

Выводы:
- явных повторов строк не обнаружено
- количество стран в колонке `language` - 85
- информация о странах внесена менее, чем в 30% записей
- все `track_id` уникальны
- даты представлены в unix формате
- можно удалить признаки:
  - `duration` - не несёт важной информации, т.к. продолжительность кавера может отличаться от исходника
  - `genres` - не несёт важной информации, т.к. жанр кавера может отличаться от исходника
  - `isrc` - не несёт важной информации, т.к. имеется признак `track_id`, а ещё в нём есть пропуски

### Датафрейм `covers`

In [8]:
covers.info()
display(covers.sample())
print('явные повторы строк:', covers.duplicated().sum())
display('количество уникальных записей:', covers.nunique())
print('оригинальных треков указано:', round(covers['original_track_id'].count() / covers['track_id'].count() * 100, 2), '% записей')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71597 entries, 0 to 71596
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_track_id  4821 non-null   object
 1   track_id           71597 non-null  object
 2   track_remake_type  71597 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


,original_track_id,track_id,track_remake_type
54987,None,046233b0a4583c0eac256374c550cf8b,COVER


явные повторы строк: 0


'количество уникальных записей:'

original_track_id     4468
track_id             71597
track_remake_type        2
dtype: int64

оригинальных треков указано: 6.73 % записей


In [9]:
# удаление пропусков в целевом признаке
filled_data = covers.dropna(subset=['track_remake_type'])
filled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71597 entries, 0 to 71596
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_track_id  4821 non-null   object
 1   track_id           71597 non-null  object
 2   track_remake_type  71597 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


Выводы:
- явных повторов строк не обнаружено
- количество указанных оригинальных треков - 6.73 % от общего числа треков
- удалены пропуски в целевом признаке

### Объединение датафреймов

In [10]:
cover_lyric = (filled_data
               .merge(lyrics, on='track_id', how='left')
               .merge(meta[['track_id', 'dttm']], on='track_id', how='left')
              ) # объединение по левой таблице
cover_lyric_wona = cover_lyric.dropna(subset=['text', 'original_track_id']).reset_index(drop=True) # удаление пропусков
cover_lyric_wona = cover_lyric_wona.drop(['lyricId'], axis=1) # удаление колонки lyricId

cover_lyric_wona.shape

(3599, 5)

Выводы:
- к основному датафрейму присоединены дополнительные
- из объединённого датафрейма удалены пропуски и ненужные признаки
- в данных осталось 3599 записей

### Кодирование текстов

In [11]:
sentences = np.array(cover_lyric_wona['text'])

model = SentenceTransformer('all-MiniLM-L6-v2')  # new
# model = SentenceTransformer('sentence-transformers/LaBSE')  # old

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(3599, 384)

Вывод:
- тексты переведены в векторы при помощи `SentenceTransformer`

Соединение закодированных текстов с исходными данными

In [12]:
df_work = cover_lyric_wona.copy()
df_work = df_work.join(pd.DataFrame(sentence_embeddings), how='left')
df_work.head()

,original_track_id,track_id,track_remake_type,text,dttm,0,1,2,3,4,...,374,375,376,377,378,379,380,381,382,383
0,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL,Left a good job in the city\nWorkin' for the m...,1.257973e+12,-0.036484,-0.010698,0.029330,-0.011374,-0.033324,...,0.098189,-0.027797,0.002791,0.039005,-0.001801,0.005720,-0.031388,0.068490,0.019620,-0.073913
1,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL,Left a good job in the city\nWorkin' for the m...,1.257973e+12,-0.031461,-0.019154,0.035092,-0.018275,-0.034330,...,0.098580,-0.021681,0.002494,0.030693,-0.006861,0.007400,-0.035143,0.060066,0.027317,-0.070681
2,fe7ee8fc1959cc7214fa21c4840dff0a,fe7ee8fc1959cc7214fa21c4840dff0a,ORIGINAL,Some folks are born made to wave the flag\nOoh...,1.257973e+12,-0.053880,0.012169,0.019933,-0.073982,-0.022932,...,0.032407,-0.043861,0.029677,0.077275,0.032256,-0.025936,0.044958,-0.008292,0.023528,-0.085016
3,cd89fef7ffdd490db800357f47722b20,cd89fef7ffdd490db800357f47722b20,ORIGINAL,"Uno por pobre y feo, hombre\nPero antoja'o, ay...",1.253563e+12,-0.080533,0.020840,0.062667,-0.047301,0.036674,...,-0.028124,-0.088308,0.096382,0.048526,-0.036241,0.010646,0.094440,0.083824,-0.027932,-0.106751
4,995665640dc319973d3173a74a03860c,995665640dc319973d3173a74a03860c,ORIGINAL,"Yeah!... yeah!... remember the time, baby... y...",1.258405e+12,-0.112342,0.004634,0.046857,-0.000905,0.012664,...,0.106143,-0.036825,0.032805,0.052460,0.039913,0.011443,-0.006147,0.010955,0.059665,-0.152829


Вывод:
- к датафрейму присоединены векторные значения текстов

### Подготовка к обучению

In [13]:
'соотношение классов:', round(df_work['track_remake_type'].value_counts()[1] / df_work['track_remake_type'].value_counts()[0], 2)

('соотношение классов:', 0.12)

Вывод:
- в данных наблюдается значительный дисбаланс классов
- при разделении на выборки применим параметр `stratify`

In [14]:
df_tr = df_work.copy()
X = df_tr.drop(['original_track_id', 'track_id', 'track_remake_type', 'text'], axis=1)
y = df_tr['track_remake_type']
y = (y != 'ORIGINAL').astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=RS,
                                                    stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2879, 385), (720, 385), (2879,), (720,))

Вывод:
- данные разделены на выборки с учётом дисбаланса классов

## Model

In [15]:
# Определение функции оценки F1-меры
def f1_eval(preds, dtrain):
    labels = dtrain.get_label()
    preds_binary = [1 if p >= 0.5 else 0 for p in preds]
    return 'f1', f1_score(labels, preds_binary)

In [16]:
# Параметры модели
params = {
    'objective':'binary:logistic',
    'eval_metric': 'logloss', #f1_eval,  # Использование своей функции оценки F1-меры
    'n_estimators': 2,
    'max_depth': 2,
    'learning_rate': 0.5
}

# Преобразование данных в формат DMatrix
dtrain = xgboost.DMatrix(X_train, label=y_train)
dtest = xgboost.DMatrix(X_test, label=y_test)

# Обучение модели
model = xgboost.train(params, dtrain, num_boost_round=100)

# Прогнозирование на тестовом наборе данных
y_pred = model.predict(dtest)
y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred_binary)
print("F1-мера:", f1)

F1-мера: 0.6356589147286822


Вывод:
- при обучении `XGBoost` получено значение `f1 = 0.636`